In [ ]:
import pandas as pd
import numpy as np
import gc
import warnings
warnings.filterwarnings("ignore")

In [ ]:
X = pd.read_csv("/content/Preprocessed_X_data.csv").drop("Unnamed: 0",axis=1)
y = pd.read_csv("/content/Preprocessed_y_data.csv").drop("Unnamed: 0",axis=1)
feature_name = X.columns.tolist()

In [ ]:
selected_features = pd.read_csv("/content/Selected_Features_Breast_Cancer.csv").drop("Unnamed: 0",axis=1)

In [ ]:
selected_features

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,Total
0,texture_worst,True,True,True,True,True,True,6
1,texture_mean,True,True,True,True,True,True,6
2,radius_worst,True,True,True,True,True,True,6
3,perimeter_worst,True,True,True,True,True,True,6
4,perimeter_mean,True,True,True,True,True,True,6
5,concavity_mean,True,True,True,True,True,True,6
6,concave points_worst,True,True,True,True,True,True,6
7,concave points_mean,True,True,True,True,True,True,6
8,radius_mean,True,True,True,True,True,False,5
9,concavity_worst,True,True,True,False,True,True,5


In [ ]:
y=y['diagnosis']

In [ ]:
from hpsklearn import HyperoptEstimator
from hpsklearn import any_classifier
from hpsklearn import any_preprocessing
from hyperopt import tpe

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [ ]:
X.values

array([[0.52103744, 0.0226581 , 0.54598853, ..., 0.91202749, 0.59846245,
        0.41886396],
       [0.64314449, 0.27257355, 0.61578329, ..., 0.63917526, 0.23358959,
        0.22287813],
       [0.60149557, 0.3902604 , 0.59574321, ..., 0.83505155, 0.40370589,
        0.21343303],
       ...,
       [0.32517272, 0.43579614, 0.34447014, ..., 0.69331042, 0.3605019 ,
        0.48959748],
       [0.41282336, 0.35540163, 0.43525458, ..., 0.74033367, 0.52559164,
        0.34833193],
       [0.39424283, 0.29649177, 0.41067021, ..., 0.76992605, 0.34854103,
        0.43310644]])

In [ ]:
y

0      1
1      1
2      1
3      1
4      1
      ..
710    1
711    1
712    1
713    1
714    1
Name: diagnosis, Length: 715, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=1)

In [ ]:
X_train.shape

(572, 30)

In [ ]:
y_train.shape

(572,)

In [ ]:
!pip install git+https://github.com/hyperopt/hyperopt-sklearn.git

In [ ]:
import xgboost as xgb

In [ ]:
from hpsklearn import HyperoptEstimator, svc
from sklearn import svm
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
import xgboost as xgb

In [ ]:
def objective(space):

    warnings.filterwarnings(action='ignore', category=DeprecationWarning)
    classifier = xgb.XGBClassifier(n_estimators = space['n_estimators'],
                            max_depth = int(space['max_depth']),
                            learning_rate = space['learning_rate'],
                            gamma = space['gamma'],
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                            colsample_bytree = space['colsample_bytree']
                            )
    
    classifier.fit(X.values, y.values)

    # Applying k-Fold Cross Validation
    from sklearn.model_selection import cross_val_score
    accuracies = cross_val_score(estimator = classifier, X = X.values, y = y.values, cv = 10)
    CrossValMean = accuracies.mean()

    print("CrossValMean:", CrossValMean)

    return{'loss':1-CrossValMean, 'status': STATUS_OK }

In [ ]:
space = {
    'max_depth' : hp.choice('max_depth', range(5, 30, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.01),
    'n_estimators' : hp.choice('n_estimators', range(20, 205, 5)),
    'gamma' : hp.quniform('gamma', 0, 0.50, 0.01),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.01),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.01)}

In [ ]:
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)

CrossValMean:
0.9552230046948356
CrossValMean:
0.9635758998435054
CrossValMean:
0.9299882629107981
CrossValMean:
0.9636150234741784
CrossValMean:
0.9523865414710484
CrossValMean:
0.9580203442879499
CrossValMean:
0.9440140845070422
CrossValMean:
0.9650234741784038
CrossValMean:
0.9538145539906104
CrossValMean:
0.9664319248826292
CrossValMean:
0.9692488262910798
CrossValMean:
0.962206572769953
CrossValMean:
0.9608372456964007
CrossValMean:
0.9705790297339594
CrossValMean:
0.9734154929577464
CrossValMean:
0.9509976525821596
CrossValMean:
0.9748043818466353
CrossValMean:
0.960817683881064
CrossValMean:
0.941255868544601
CrossValMean:
0.9748435054773082
CrossValMean:
0.9705985915492958
CrossValMean:
0.946830985915493
CrossValMean:
0.9720266040688574
CrossValMean:
0.9734154929577464
CrossValMean:
0.9565923317683882
CrossValMean:
0.9580594679186228
CrossValMean:
0.974823943661972
CrossValMean:
0.9692292644757433
CrossValMean:
0.9467527386541471
CrossValMean:
0.9678208137715181
CrossValMean:
0

In [ ]:
print("Best: ", best)


Best:  {'colsample_bytree': 0.86, 'gamma': 0.08, 'learning_rate': 0.38, 'max_depth': 22, 'min_child_weight': 1.0, 'n_estimators': 8, 'subsample': 0.8300000000000001}


In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier(n_estimators = best['n_estimators'],
                            max_depth = best['max_depth'],
                            learning_rate = best['learning_rate'],
                            gamma = best['gamma'],
                            min_child_weight = best['min_child_weight'],
                            subsample = best['subsample'],
                            colsample_bytree = best['colsample_bytree']
                            )

classifier.fit(X.values, y.values)

# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X.values, y = y.values, cv = 10)
CrossValMean = accuracies.mean()
print("Final CrossValMean: ", CrossValMean)

Final CrossValMean:  0.9621870109546167
